In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
import pandas as pd

from gen_planner.python.src import GenPlanner,func_zones, gen_plan

zones, roads = GenPlanner(gpd.read_file('1.gpkg'), rotation=0).terr2district2zone2block(gen_plan)


In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [2]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from gen_planner.python.src import GenPlanner,basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).district2zone2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
res, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from zoning import func_zones, basic_scenario

res, roads = GenPlanner(gpd.read_file('светогорск.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from zoning import terr_zones

res, roads = GenPlanner(gpd.read_file('светогорск.geojson')).zone2block(terr_zones.residential)
res.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2')